In [ ]:
# ==============================================
# STEP 1: Mount Google Drive
# ==============================================
from google.colab import drive
drive.mount('/content/gdrive')

# Define root path
ROOT_DIR = '/content/gdrive/MyDrive/MPfinal'


# ==============================================
# STEP 2: Install and Import YOLOv8 (Ultralytics)
# ==============================================
!pip install ultralytics --quiet

import os
from ultralytics import YOLO
import pandas as pd
import re


# ==============================================
# STEP 3: Dataset structure check (Informational)
# ==============================================
# Your directory must look like:
# MPfinal/
#   ├── train/
#   │     ├── images/
#   │     └── labels/
#   ├── valid/
#   │     ├── images/
#   │     └── labels/
#   └── data.yaml
#
# Example content of data.yaml (ensure class names match your labels):
# train: /content/gdrive/My Drive/MPfinal/train/images
# val: /content/gdrive/My Drive/MPfinal/valid/images
# names:
#   0: attentive
#   1: non_attentive
# ==============================================


# ==============================================
# STEP 4: Load pretrained model (transfer learning)
# ==============================================
model = YOLO("yolov8s.pt")  # pre-trained model


# ==============================================
# STEP 5: Train the model
# ==============================================
print("🚀 Starting Model Training...")
results = model.train(
    data=os.path.join(ROOT_DIR, "data.yaml"),
    epochs=200,
    batch=16,
    imgsz=640,
    lr0=0.001,
    optimizer='Adam',
    pretrained=True,
    patience=20,
    augment=True,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=10.0, translate=0.1, scale=0.5, shear=2.0,
    flipud=0.2, fliplr=0.5,
)
print("✅ Training Complete.")


# ==============================================
# STEP 6 & 10: Validate model (Run once after training)
# ==============================================
print("\n🔍 Validating Model...")
metrics = model.val()
print("📊 Validation metrics:")
# print(metrics) # Kept commented to avoid verbose raw output clutter, relying on summary below

# Display Key Accuracy Results in Colab Output (with rating)
precision = metrics.box.mp        # mean precision
recall = metrics.box.mr           # mean recall
map50 = metrics.box.map50         # mAP@0.5
map5095 = metrics.box.map         # mAP@0.5:0.95

# Assign emoji-based rating for quick interpretation
if map50 >= 0.9:
    rating = "🌟 Excellent"
elif map50 >= 0.8:
    rating = "✅ Good"
elif map50 >= 0.7:
    rating = "⚠️ Fair"
else:
    rating = "🔴 Needs Improvement"

# Print a nicely formatted summary
print("\n===============================")
print("📈 MODEL ACCURACY SUMMARY")
print("===============================")
print(f"🔹 Precision:           {precision:.3f}")
print(f"🔹 Recall:              {recall:.3f}")
print(f"🔹 mAP@0.5:             {map50:.3f}")
print(f"🔹 mAP@0.5:0.95:       {map5095:.3f}")
print(f"\n🏁 Overall Rating:      {rating}")
print("===============================")


# ==============================================
# STEP 7: Make predictions
# ==============================================
print("\n🔮 Making Predictions on Validation Set...")
preds = model.predict(
    source=os.path.join(ROOT_DIR, "valid/images"),
    save=True,
    conf=0.4, # This threshold only affects the *saved* prediction images/txt, not the validation metrics above
    iou=0.45
)
print("✅ Predictions Generated.")


# ==============================================
# STEP 8: Generate Corrected CSV report (PRN + confidence)
# ==============================================
print("\n📄 Generating Corrected CSV Report...")
results_list = []

for r in preds:
    image_name = os.path.basename(r.path)

    # Extract PRN (assumes filename contains digits like 49, 57, 70, 81, 90...)
    prn_match = re.search(r'(\d+)', image_name)
    prn = prn_match.group(1) if prn_match else "UNKNOWN"

    if not hasattr(r, 'boxes'):
        continue

    # Only log the true predicted class and its confidence score (box.conf)
    for box in r.boxes:
        conf = float(box.conf)
        label = model.names[int(box.cls)]

        # We only log the winning class and its confidence score, removing the incorrect secondary score calculation
        results_list.append({
            "PRN": prn,
            "Predicted_Class": label,
            "Confidence_Score": round(conf, 3), # This is the max score for the 'Predicted_Class'
        })

# Create DataFrame
df = pd.DataFrame(results_list)

# Save to CSV
csv_path = os.path.join(ROOT_DIR, "student_attendance_predictions_corrected.csv")
df.to_csv(csv_path, index=False)

print(f"✅ Corrected CSV file saved at: {csv_path}")
print("\n--- CSV Head (First 5 Rows) ---")
print(df.head())
print("-------------------------------\n")


# ==============================================
# STEP 9: Backup training results
# ==============================================
print("💾 Backing up training results...")
# Note: Using 'runs/detect/train*' might be safer if you run multiple training sessions
!cp -r /content/runs/detect/train* /content/gdrive/MyDrive/MPfinal/runs_backup
print("📂 Training runs backed up successfully to Google Drive!")
# ==============================================

Mounted at /content/gdrive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Starting Model Training...
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/MPfinal/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5

In [ ]:
# ==============================================
# FIX: Re-import necessary modules for Step 11
# ==============================================
from google.colab import drive
drive.mount('/content/gdrive')

import os # <-- FIXING THE NameError HERE!
import pandas as pd
import re
from ultralytics import YOLO
# Note: Since you already trained and saved the model,
# we'll load it from the saved 'best.pt' file instead of reloading 'yolov8s.pt'.

# Define root path again for context
ROOT_DIR = '/content/gdrive/MyDrive/MPfinal'

print("✅ Essential modules and environment variables reloaded.")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
✅ Essential modules and environment variables reloaded.


In [ ]:
# ==============================================
# STEP 11: Predict on an Input Video from Google Drive (MEMORY OPTIMIZED)
# ==============================================

# --- 1. DEFINE YOUR INPUT VIDEO PATH HERE ---
# IMPORTANT: Change this path to the actual location of your video file in Google Drive!
VIDEO_SOURCE_PATH = '/content/gdrive/MyDrive/video_1.mp4' # <<< CHANGE THIS

PREDICTION_SAVE_DIR = os.path.join(ROOT_DIR, "video_predictions")
os.makedirs(PREDICTION_SAVE_DIR, exist_ok=True)

print(f"🔮 Starting video prediction on: {VIDEO_SOURCE_PATH}")
print(f"💾 Saving results to: {PREDICTION_SAVE_DIR}")

# --- Model Loading Logic ---
# Try loading the best model from your training run first
try:
    best_model_path = None
    # Search for the latest training run folder
    run_dirs = sorted(os.listdir('/content/runs/detect/'), reverse=True)
    for d in run_dirs:
        potential_path = os.path.join('/content/runs/detect', d, 'weights', 'best.pt')
        if os.path.exists(potential_path):
            best_model_path = potential_path
            break

    if best_model_path:
        print(f"   > Loading custom trained model from: {best_model_path}")
        model_to_use = YOLO(best_model_path)
    else:
        # If best.pt isn't found, load the baseline model you used in your script (yolov8s.pt)
        print("   > Custom 'best.pt' not found. Loading base YOLOv8s model.")
        model_to_use = YOLO("yolov8s.pt")

except Exception as e:
    print(f"   > Error during model loading fallback: {e}. Using base YOLOv8s model.")
    model_to_use = YOLO("yolov8s.pt")

# --- MEMORY OPTIMIZATION ---
# Process every 10th frame to conserve RAM/VRAM. Adjust higher if needed.
stride_value = 10
print(f"   > Using vid_stride={stride_value} to skip frames for lower memory usage.")

# Run prediction on the video source
prediction_results = model_to_use.predict(
    source=VIDEO_SOURCE_PATH,
    save=True,           # Save the output video
    conf=0.4,
    iou=0.45,
    save_dir=PREDICTION_SAVE_DIR,
    vid_stride=stride_value
)

print("\n✅ Video Prediction Complete!")
# ==============================================

🔮 Starting video prediction on: /content/gdrive/MyDrive/video_1.mp4
💾 Saving results to: /content/gdrive/MyDrive/MPfinal/video_predictions
   > Loading custom trained model from: /content/runs/detect/train/weights/best.pt
   > Using vid_stride=10 to skip frames for lower memory usage.

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/226) /content/gdrive/MyDrive/video_1.mp4: 384x640 1 22510001_non_attentive, 1 22510002_attentive, 1 22510003_attentive, 1 22510004_non_attentive, 1 225